import

In [1]:
import matplotlib.pyplot as plt
import torch
import os
import pandas as pd
import math

from tqdm.notebook import tqdm as tqdm
from tqdm.auto import trange

import warnings
warnings.filterwarnings("ignore")

from myhelpers import config_plots, TrialStatistics
from HGNN.train import CNN, dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from flashtorch_modefied import CNN_wrapper, Backprop, SaliencyMap

config_plots.global_settings()

experimetnsFileName = "experiments.csv"

parameters

In [2]:
experimentsPath="/raid/elhamod/Fish/experiments/"
dataPath="/raid/elhamod/Fish"
experimentName="biology_paper_easy_curated4_Easy_smaller"
# trial_hash="5b37447d8a18a795d2d6da930c9783a47d75cbf4e7a4d00981e5bded" # BB
trial_hash='29ff9b40ebac1251756a4b178d19c9e4c8cd2124f4ecde33f5b52edb'

num_workers=8
patchsize=49

cuda=6

cuda

In [3]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

using cuda 6


importLoad experiment

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

experimentPathAndName = os.path.join(experimentsPath, experimentName)

datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  

load dataset and model

In [5]:
# Get experiment parameters
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
else:
    raise Exception("Experiment not " + trial_hash + " found!")
experimentRecord = experiments_df[experiments_df["trialHash"] == trial_hash]
experiment_params = experimentRecord.to_dict('records')[0]
if math.isnan(experiment_params['suffix']):
    experiment_params['suffix'] = None
print(experiment_params)


# Create the model
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
train_loader, validation_loader, test_loader = datasetManager.getLoaders()
fineList = test_loader.dataset.csv_processor.getFineList()
coarseList = test_loader.dataset.csv_processor.getCoarseList()
numberOffine = len(fineList)
numberOfcoarse = len(coarseList)
architecture = {
    "fine": numberOffine,
    "coarse" : numberOfcoarse
}
model = CNN.create_model(architecture, experiment_params, device=cuda)

{'experimentName': 'biology_paper_easy_curated4_Easy_smaller', 'modelName': 'models/29ff9b40ebac1251756a4b178d19c9e4c8cd2124f4ecde33f5b52edb', 'datasetName': 'datasplits/7c7513bdfb4e5577fef1c3ec0fa6452d30d87dbc95f258a7c86dd76b', 'experimentHash': 'a34fd211dd815588f94bc4b82f71091d824df75b0379c883de362994', 'trialHash': '29ff9b40ebac1251756a4b178d19c9e4c8cd2124f4ecde33f5b52edb', 'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.0001, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.01, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.01, 'adaptive_alpha': 0.9}
Creating datasets...
Creating datasets... Done.
Creating loaders...
Creating loaders... Done.


Load model of a specific trial

In [6]:
# get the model and the parameters
modelName = experimentRecord.iloc[0]["modelName"]
trialName = os.path.join(experimentPathAndName, modelName)

df, epochs, time_elapsed = CNN.loadModel(model, trialName, device=cuda)

Model wrapper

In [7]:
wrapped_model = CNN_wrapper(model, experiment_params, test_loader.dataset)
saliencyMap = SaliencyMap(test_loader.dataset, wrapped_model, experimentPathAndName, trial_hash, experiment_params)

Iterate and give score

In [8]:
from PIL import Image

import gc


def getAverageCorrectProb(loader, layerName, box_width, topk, df):
    prob = []
    with tqdm(total=len(loader.dataset)) as bar:
        colName = layerName + "/box" + "/iter" + str(topk)
        df[colName, "prob"] = ""
        df[colName, "predictedFine"] = ""
        df[colName, "withinSameGenus"] = ""
        for img in loader:
            fileNames = img['fileNameFull']
            fileName_bases =  img['fileName']
            lbls=img[layerName]
            
            for i, fileName in enumerate(fileNames):
                fileName_base = fileName_bases[i]
                if topk > 0:
                    _, A = saliencyMap.GetSaliencyMap(fileName,layerName,box_width =box_width, maxCovered=True, topk=topk, plot=False, use_gpu=True,generate_all_steps=False)
                else:
                    isSpecies = (layerName == "fine")
                    activationOutputs = {
                        "fine": isSpecies,
                        "coarse" : not isSpecies
                    }
                    img = saliencyMap.getTransformedImage(Image.open(fileName), False, True)
                    wrapped_model.setOutputsOfInterest(activationOutputs)

                    if torch.cuda.is_available():
                        img = img.cuda()
                    A = wrapped_model(img)
                    
                if (layerName == "coarse" or layerName == "fine"):
                    A = torch.nn.Softmax(dim=1)(A)
                
                if torch.cuda.is_available():
                    A = A.detach().cpu()
                prob_ = A[0][lbls[i]]
                prob.append(prob_)

                fine = loader.dataset.csv_processor.getFineLabel(fileName_base)
                coarse = loader.dataset.csv_processor.getCoarseLabel(fileName_base)
                pred_fine = loader.dataset.csv_processor.getFineList()[torch.argmax(A[0])] 
                right_genus = loader.dataset.csv_processor.getCoarseFromFine(pred_fine) == loader.dataset.csv_processor.getCoarseFromFine(fine)
                
                if fileName_base not in df.index: 
                    temp = pd.DataFrame({
                        "fileName": [fileName_base],
                        "fine": [fine],
                        "coarse": [coarse],
                    })
                    temp = temp.set_index( "fileName")
                    df = df.append(temp)
                
                df.loc[fileName_base][colName, "prob"] = prob_.item()
                df.loc[fileName_base][colName, "predictedFine"] = pred_fine
                df.loc[fileName_base][colName, "withinSameGenus"] = right_genus

                bar.update()
                

    return (sum(prob) / len(prob)).item(), df


def add_occlusion_result(df, loader, func, label, patchsize, iterations, df2):
    result, df2 = func(loader, label, patchsize, iterations, df2)
    return df.append(pd.DataFrame({
        "label": [label],
        "patch size": [patchsize],
        "iterations": [iterations],
        "average correct probability": [result]
    })), df2

In [9]:
# Create the test loader with small batch
test_loader = torch.utils.data.DataLoader(test_loader.dataset, batch_size=100, num_workers=num_workers)

df = pd.DataFrame()
df2 = pd.DataFrame(columns=['fileName', 'fine', 'coarse'])
df2 = df2.set_index('fileName')

In [10]:
df, df2 = add_occlusion_result(df, test_loader, getAverageCorrectProb, "fine", patchsize, 0, df2)

In [11]:
df2

,fine,coarse,"(fine/box/iter0, prob)","(fine/box/iter0, predictedFine)","(fine/box/iter0, withinSameGenus)"
fileName,,,,,
INHS_FISH_008.JPG,Alosa chrysochloris,Alosa,0.971997,Alosa chrysochloris,True
INHS_FISH_103372.JPG,Alosa chrysochloris,Alosa,0.789335,Alosa chrysochloris,True
INHS_FISH_105918.JPG,Alosa chrysochloris,Alosa,0.854159,Alosa chrysochloris,True
INHS_FISH_106186.JPG,Alosa chrysochloris,Alosa,0.946035,Alosa chrysochloris,True
INHS_FISH_106912.JPG,Alosa chrysochloris,Alosa,0.692997,Alosa chrysochloris,True
...,...,...,...,...,...
INHS_FISH_73996.jpg,Phenacobius mirabilis,Phenacobius,0.868398,Phenacobius mirabilis,True
INHS_FISH_8554.jpg,Phenacobius mirabilis,Phenacobius,0.825556,Phenacobius mirabilis,True
INHS_FISH_86624.jpg,Phenacobius mirabilis,Phenacobius,0.731294,Phenacobius mirabilis,True


In [12]:
df, df2 = add_occlusion_result(df, test_loader, getAverageCorrectProb, "fine", patchsize, 1, df2)

In [13]:
# df = df.append(pd.DataFrame({
#         "label": ["test"],
#         "patch size": [19],
#         "iterations": [1],
#         "average correct probability": [1.23]
#     }))

In [14]:
df, df2 = add_occlusion_result(df, test_loader, getAverageCorrectProb, "fine", patchsize, 2, df2)

In [15]:
# df, df2= add_occlusion_result(df, test_loader, getAverageCorrectProb, "coarse", patchsize, 0, df2)
df, df2 = add_occlusion_result(df, test_loader, getAverageCorrectProb, "fine", patchsize, 3, df2)

In [16]:
# df, df2= add_occlusion_result(df, test_loader, getAverageCorrectProb, "coarse", patchsize, 1, df2)
df, df2 = add_occlusion_result(df, test_loader, getAverageCorrectProb, "fine", patchsize, 4, df2)

In [17]:
# df, df2 = add_occlusion_result(df, test_loader, getAverageCorrectProb, "coarse", patchsize, 2, df2)

In [18]:
df.reset_index().to_csv(os.path.join(experimentPathAndName, "models", trial_hash, 'occlusion_summary.csv'))

In [19]:
df

,label,patch size,iterations,average correct probability
0,fine,49,0,0.472690
0,fine,49,1,0.355305
0,fine,49,2,0.291387
0,fine,49,3,0.232104
0,fine,49,4,0.187159


In [20]:
df2.reset_index().to_csv(os.path.join(experimentPathAndName, "models", trial_hash, 'occlusion_perfile.csv'))

In [21]:
df2

,fine,coarse,"(fine/box/iter0, prob)","(fine/box/iter0, predictedFine)","(fine/box/iter0, withinSameGenus)","(fine/box/iter1, prob)","(fine/box/iter1, predictedFine)","(fine/box/iter1, withinSameGenus)","(fine/box/iter2, prob)","(fine/box/iter2, predictedFine)","(fine/box/iter2, withinSameGenus)","(fine/box/iter3, prob)","(fine/box/iter3, predictedFine)","(fine/box/iter3, withinSameGenus)","(fine/box/iter4, prob)","(fine/box/iter4, predictedFine)","(fine/box/iter4, withinSameGenus)"
fileName,,,,,,,,,,,,,,,,,
INHS_FISH_008.JPG,Alosa chrysochloris,Alosa,0.971997,Alosa chrysochloris,True,0.841685,Alosa chrysochloris,True,0.766722,Alosa chrysochloris,True,0.777102,Alosa chrysochloris,True,0.565559,Alosa chrysochloris,True
INHS_FISH_103372.JPG,Alosa chrysochloris,Alosa,0.789335,Alosa chrysochloris,True,0.728525,Alosa chrysochloris,True,0.6221,Alosa chrysochloris,True,0.562806,Alosa chrysochloris,True,0.589084,Alosa chrysochloris,True
INHS_FISH_105918.JPG,Alosa chrysochloris,Alosa,0.854159,Alosa chrysochloris,True,0.681441,Alosa chrysochloris,True,0.528045,Alosa chrysochloris,True,0.329239,Alosa chrysochloris,True,0.217076,Alosa chrysochloris,True
INHS_FISH_106186.JPG,Alosa chrysochloris,Alosa,0.946035,Alosa chrysochloris,True,0.937149,Alosa chrysochloris,True,0.818392,Alosa chrysochloris,True,0.578674,Alosa chrysochloris,True,0.45517,Alosa chrysochloris,True
INHS_FISH_106912.JPG,Alosa chrysochloris,Alosa,0.692997,Alosa chrysochloris,True,0.615626,Alosa chrysochloris,True,0.518087,Alosa chrysochloris,True,0.524246,Alosa chrysochloris,True,0.456014,Alosa chrysochloris,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INHS_FISH_73996.jpg,Phenacobius mirabilis,Phenacobius,0.868398,Phenacobius mirabilis,True,0.599024,Phenacobius mirabilis,True,0.51719,Phenacobius mirabilis,True,0.0702017,Notropis blennius,False,0.0748847,Notropis blennius,False
INHS_FISH_8554.jpg,Phenacobius mirabilis,Phenacobius,0.825556,Phenacobius mirabilis,True,0.471271,Phenacobius mirabilis,True,0.45666,Phenacobius mirabilis,True,0.046675,Notropis blennius,False,0.0507741,Notropis blennius,False
INHS_FISH_86624.jpg,Phenacobius mirabilis,Phenacobius,0.731294,Phenacobius mirabilis,True,0.281616,Phenacobius mirabilis,True,0.29495,Phenacobius mirabilis,True,0.0566986,Noturus nocturnus,False,0.0616829,Phenacobius mirabilis,True
